# Structuring: Statistical Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import entropy
from cliffs_delta import cliffs_delta
import warnings
warnings.filterwarnings('ignore')

# Set display options for better table formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Set style for better-looking plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

#from statsmodels.stats.inter_rater import fleiss_kappa

### Data Loading and Preprocessing

In [2]:
# Read the CSV file with semicolon separator
df = pd.read_csv('Structuring-data.csv', sep=';')

# Define question columns
question_cols = [col for col in df.columns if col.startswith('D')]

# Convert question columns to numeric, handling missing values
for col in question_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Display basic info about the dataset
print(f"Dataset shape: {df.shape}")
print(f"\nNumber of determinant questions: {len(question_cols)}")

Dataset shape: (90, 33)

Number of determinant questions: 29


### Auxiliary Methods

In [3]:
def calculate_rwg_agreement(data):
    """Calculate rwg (within-group inter-rater agreement) statistic
    
    rwg = 1 - (Observed Group Variance / Expected Random Variance)
    
    Expected Random Variance for 7-point scale = (7^2 - 1) / 12 = 4.0
    """
    
    # Remove missing values
    valid_data = data.dropna()
    
    if len(valid_data) == 0:
        return np.nan
    
    # Expected random variance for 7-point scale (1-7)
    # Formula: (k^2 - 1) / 12 where k = number of scale points
    expected_random_variance = (7**2 - 1) / 12  # = 4.0
    
    # Observed group variance
    observed_variance = np.var(valid_data, ddof=1)
    
    # Calculate rwg
    if expected_random_variance == 0:
        rwg = np.nan
    else:
        rwg = 1 - (observed_variance / expected_random_variance)
    
    return rwg

def calculate_entropy(data):
    """Calculate entropy for categorical data"""
    if len(data.dropna()) == 0:
        return np.nan
    
    # Count frequencies of each value
    value_counts = data.value_counts()
    # Calculate probabilities
    probabilities = value_counts / len(data.dropna())
    # Calculate entropy
    return entropy(probabilities)

def calculate_inter_rater_agreement(data):
    """Calculate inter-rater agreement using Fleiss' Kappa for multiple raters"""
    
    # Remove missing values
    valid_data = data.dropna()
    
    if len(valid_data) == 0:
        return np.nan
    
    # Calculate Fleiss' Kappa manually
    n_raters = len(valid_data)
    n_categories = 7
    
    # Count how many raters chose each category
    category_counts = np.zeros(n_categories)
    for rating in valid_data:
        if 1 <= rating <= 7:
            category_counts[int(rating) - 1] += 1
    
    # Calculate observed agreement (Po)
    # Po = (sum of nij * (nij - 1)) / (N * (N-1))
    observed_agreement = np.sum(category_counts * (category_counts - 1)) / (n_raters * (n_raters - 1))
    
    # Calculate expected agreement (Pe)
    # Pe = sum of (pi^2) where pi = proportion of ratings in category i
    proportions = category_counts / n_raters
    expected_agreement = np.sum(proportions ** 2)
    
    # Calculate Fleiss' Kappa
    if expected_agreement == 1:
        kappa = 1.0
    else:
        kappa = (observed_agreement - expected_agreement) / (1 - expected_agreement)
    
    return kappa

def calculate_statistics(data):
    """Calculate comprehensive statistics for a dataset"""
    valid_data = data.dropna()
    
    if len(valid_data) == 0:
        return {
            'mean': np.nan, 'median': np.nan, 'mode': np.nan,
            'q1': np.nan, 'q3': np.nan, 'variance': np.nan,
            'skewness': np.nan, 'kurtosis': np.nan, 'entropy': np.nan,
            'count': 0
        }
    
    # Basic statistics
    mean_val = np.mean(valid_data)
    median_val = np.median(valid_data)
    mode_val = stats.mode(valid_data, keepdims=True)[0][0] if len(valid_data) > 0 else np.nan
    
    # Quartiles
    q1_val = np.percentile(valid_data, 25)
    q3_val = np.percentile(valid_data, 75)
    
    # Variance
    variance_val = np.var(valid_data, ddof=1)

    # Standard deviation
    std_dev_val = np.std(valid_data, ddof=1)
    
    # Skewness and Kurtosis
    skewness_val = stats.skew(valid_data) if len(valid_data) > 2 else np.nan
    kurtosis_val = stats.kurtosis(valid_data) if len(valid_data) > 3 else np.nan
    
    # Entropy
    entropy_val = calculate_entropy(valid_data)

    # Inter-rater agreement
    inter_rater_agreement_val = calculate_inter_rater_agreement(valid_data)

    # rwg agreement
    rwg_agreement_val = calculate_rwg_agreement(valid_data)
    
    return {
        'mean': mean_val,
        'median': median_val,
        'mode': mode_val,
        'q1': q1_val,
        'q3': q3_val,
        'variance': variance_val,
        'std_dev': std_dev_val,
        'skewness': skewness_val,
        'kurtosis': kurtosis_val,
        'entropy': entropy_val,
        'inter_rater_agreement': inter_rater_agreement_val,
        'rwg_agreement': rwg_agreement_val,
        'count': len(valid_data)
    }

def create_demographic_statistics(df, question_cols):
    """Create statistics for different demographic groups"""
    
    # Population level statistics
    population_stats = {}
    for col in question_cols:
        population_stats[col] = calculate_statistics(df[col])
    
    # Gender-specific statistics
    gender_stats = {}
    for gender in df['Gender'].unique():
        if pd.isna(gender):
            continue
        gender_data = df[df['Gender'] == gender]
        gender_stats[gender] = {}
        for col in question_cols:
            gender_stats[gender][col] = calculate_statistics(gender_data[col])
    
    # Age group statistics
    age_stats = {}
    # Create age groups: 12-13, 14-15, 16-17
    df['Age_Group'] = pd.cut(df['Age'], bins=[11, 14, 17], labels=['12-14', '15-17'])
    
    for age_group in df['Age_Group'].unique():
        if pd.isna(age_group):
            continue
        age_data = df[df['Age_Group'] == age_group]
        age_stats[age_group] = {}
        for col in question_cols:
            age_stats[age_group][col] = calculate_statistics(age_data[col])
    
    return population_stats, gender_stats, age_stats

def create_comprehensive_table(population_stats, gender_stats, age_stats, question_cols):
    """Create a comprehensive table with all statistics"""
    
    # Create the main dataframe
    stats_data = []
    
    for col in question_cols:
        # Population level
        pop_stats = population_stats[col]
        row = {
            'Determinant': col,
            'Group': 'Population',
            'Count': pop_stats['count'],
            'Mean': pop_stats['mean'],
            'Median': pop_stats['median'],
            'Mode': pop_stats['mode'],
            'Q1': pop_stats['q1'],
            'Q3': pop_stats['q3'],
            'Variance': pop_stats['variance'],
            'Std_Dev': pop_stats['std_dev'],
            'Skewness': pop_stats['skewness'],
            'Kurtosis': pop_stats['kurtosis'],
            'Entropy': pop_stats['entropy'],
            'Inter_Rater_Agreement': pop_stats['inter_rater_agreement'],
            'RWG_Agreement': pop_stats['rwg_agreement']
            
        }
        stats_data.append(row)
        
        # Gender specific
        for gender in gender_stats.keys():
            gender_stats_data = gender_stats[gender][col]
            row = {
                'Determinant': col,
                'Group': f'Gender: {gender}',
                'Count': gender_stats_data['count'],
                'Mean': gender_stats_data['mean'],
                'Median': gender_stats_data['median'],
                'Mode': gender_stats_data['mode'],
                'Q1': gender_stats_data['q1'],
                'Q3': gender_stats_data['q3'],
                'Variance': gender_stats_data['variance'],
                'Std_Dev': gender_stats_data['std_dev'],
                'Skewness': gender_stats_data['skewness'],
                'Kurtosis': gender_stats_data['kurtosis'],
                'Entropy': gender_stats_data['entropy'],
                'Inter_Rater_Agreement': gender_stats_data['inter_rater_agreement'],
                'RWG_Agreement': gender_stats_data['rwg_agreement']
            }
            stats_data.append(row)
        
        # Age group specific
        for age_group in age_stats.keys():
            age_stats_data = age_stats[age_group][col]
            row = {
                'Determinant': col,
                'Group': f'Age: {age_group}',
                'Count': age_stats_data['count'],
                'Mean': age_stats_data['mean'],
                'Median': age_stats_data['median'],
                'Mode': age_stats_data['mode'],
                'Q1': age_stats_data['q1'],
                'Q3': age_stats_data['q3'],
                'Variance': age_stats_data['variance'],
                'Std_Dev': age_stats_data['std_dev'],
                'Skewness': age_stats_data['skewness'],
                'Kurtosis': age_stats_data['kurtosis'],
                'Entropy': age_stats_data['entropy'],
                'Inter_Rater_Agreement': age_stats_data['inter_rater_agreement'],
                'RWG_Agreement': age_stats_data['rwg_agreement']
            }
            stats_data.append(row)
    
    return pd.DataFrame(stats_data)

### Inspect Missing Data

In [4]:
def calculate_missing_data_statistics(df, question_cols):
    """Calculate missing data statistics across all determinants"""
    
    print("MISSING DATA ANALYSIS")
    print("=" * 50)
    
    # Calculate missing data for each determinant
    missing_by_determinant = {}
    total_possible_responses = len(df)
    
    print("Missing data by determinant:")
    print("-" * 30)
    
    for col in question_cols:
        missing_count = df[col].isna().sum()
        missing_percentage = (missing_count / total_possible_responses) * 100
        missing_by_determinant[col] = {
            'missing_count': missing_count,
            'missing_percentage': missing_percentage,
            'valid_responses': total_possible_responses - missing_count
        }
        print(f"{col}: {missing_count}/{total_possible_responses} missing ({missing_percentage:.2f}%)")
    
    # Calculate overall missing data statistics
    total_possible_all = len(df) * len(question_cols)
    total_missing_all = sum([info['missing_count'] for info in missing_by_determinant.values()])
    overall_missing_percentage = (total_missing_all / total_possible_all) * 100
    
    print()
    print("OVERALL MISSING DATA SUMMARY:")
    print("-" * 30)
    print(f"Total possible responses: {total_possible_all}")
    print(f"Total missing responses: {total_missing_all}")
    print(f"Overall missing data percentage: {overall_missing_percentage:.2f}%")
    print(f"Overall valid data percentage: {100 - overall_missing_percentage:.2f}%")
    
    # Calculate missing data per participant
    missing_per_participant = df[question_cols].isna().sum(axis=1)
    participants_with_missing = (missing_per_participant > 0).sum()
    participants_complete = len(df) - participants_with_missing
    
    print()
    print("PARTICIPANT COMPLETENESS:")
    print("-" * 25)
    print(f"Participants with complete data: {participants_complete}/{len(df)} ({(participants_complete/len(df)*100):.2f}%)")
    print(f"Participants with some missing data: {participants_with_missing}/{len(df)} ({(participants_with_missing/len(df)*100):.2f}%)")
    
    # Show distribution of missing responses per participant
    print()
    print("Distribution of missing responses per participant:")
    missing_distribution = missing_per_participant.value_counts().sort_index()
    for missing_count, participant_count in missing_distribution.items():
        percentage = (participant_count / len(df)) * 100
        print(f"  {missing_count} missing responses: {participant_count} participants ({percentage:.2f}%)")
    
    return {
        'missing_by_determinant': missing_by_determinant,
        'total_possible_all': total_possible_all,
        'total_missing_all': total_missing_all,
        'overall_missing_percentage': overall_missing_percentage,
        'participants_complete': participants_complete,
        'participants_with_missing': participants_with_missing,
        'missing_per_participant': missing_per_participant
    }

# Calculate and display missing data statistics
missing_data_stats = calculate_missing_data_statistics(df, question_cols)

MISSING DATA ANALYSIS
Missing data by determinant:
------------------------------
D1: 1/90 missing (1.11%)
D2: 1/90 missing (1.11%)
D3: 0/90 missing (0.00%)
D4: 2/90 missing (2.22%)
D5: 0/90 missing (0.00%)
D6: 4/90 missing (4.44%)
D7: 4/90 missing (4.44%)
D8: 3/90 missing (3.33%)
D9: 4/90 missing (4.44%)
D10: 4/90 missing (4.44%)
D11: 4/90 missing (4.44%)
D12: 2/90 missing (2.22%)
D13: 5/90 missing (5.56%)
D14: 5/90 missing (5.56%)
D15: 5/90 missing (5.56%)
D16: 2/90 missing (2.22%)
D17: 2/90 missing (2.22%)
D18: 3/90 missing (3.33%)
D19: 3/90 missing (3.33%)
D20: 4/90 missing (4.44%)
D21: 2/90 missing (2.22%)
D22: 1/90 missing (1.11%)
D23: 4/90 missing (4.44%)
D24: 5/90 missing (5.56%)
D25: 5/90 missing (5.56%)
D26: 0/90 missing (0.00%)
D27: 1/90 missing (1.11%)
D28: 0/90 missing (0.00%)
D29: 0/90 missing (0.00%)

OVERALL MISSING DATA SUMMARY:
------------------------------
Total possible responses: 2610
Total missing responses: 76
Overall missing data percentage: 2.91%
Overall valid

## Statistical Difference Analysis - By Determinant

In [7]:
# Calculate all statistics
print("Calculating comprehensive statistics...")
population_stats, gender_stats, age_stats = create_demographic_statistics(df, question_cols)

# Create comprehensive table
comprehensive_table = create_comprehensive_table(population_stats, gender_stats, age_stats, question_cols)

print("Statistical analysis completed!")
print(f"Total rows in comprehensive table: {len(comprehensive_table)}")

Calculating comprehensive statistics...
Statistical analysis completed!
Total rows in comprehensive table: 145


In [8]:
def display_population_summary(population_stats, question_cols):
    """Display population-level statistics summary"""
    print("POPULATION-LEVEL STATISTICS SUMMARY")
    print("=" * 80)
    
    # Create summary dataframe
    summary_data = []
    for col in question_cols:
        stats = population_stats[col]
        summary_data.append({
            'Determinant': col,
            'Count': stats['count'],
            'Mean': f"{stats['mean']:.3f}",
            'Median': f"{stats['median']:.1f}",
            'Mode': f"{stats['mode']:.0f}",
            'Q1': f"{stats['q1']:.1f}",
            'Q3': f"{stats['q3']:.1f}",
            'Variance': f"{stats['variance']:.3f}",
            'Std_Dev': f"{stats['std_dev']:.3f}",
            'Skewness': f"{stats['skewness']:.3f}",
            'Kurtosis': f"{stats['kurtosis']:.3f}",
            'Entropy': f"{stats['entropy']:.3f}",
            'Inter_Rater_Agreement': f"{stats['inter_rater_agreement']:.1f}%",
            'RWG_Agreement': f"{stats['rwg_agreement']:.3f}"
        })
    
    summary_df = pd.DataFrame(summary_data)
    print(summary_df.to_string(index=False))
    print()

def display_gender_comparison(gender_stats, question_cols):
    """Display gender comparison for key statistics"""
    print("GENDER COMPARISON - KEY STATISTICS")
    print("=" * 80)
    
    comparison_data = []
    for col in question_cols:
        row = {'Determinant': col}
        for gender in gender_stats.keys():
            stats = gender_stats[gender][col]
            row[f'{gender}_Mean'] = f"{stats['mean']:.3f}"
            row[f'{gender}_Count'] = stats['count']
        comparison_data.append(row)
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))
    print()

def display_age_comparison(age_stats, question_cols):
    """Display age group comparison for key statistics"""
    print("AGE GROUP COMPARISON - KEY STATISTICS")
    print("=" * 80)
    
    comparison_data = []
    for col in question_cols:
        row = {'Determinant': col}
        for age_group in age_stats.keys():
            stats = age_stats[age_group][col]
            row[f'{age_group}_Mean'] = f"{stats['mean']:.3f}"
            row[f'{age_group}_Count'] = stats['count']
        comparison_data.append(row)
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))
    print()

# Display all summaries
display_population_summary(population_stats, question_cols)
display_gender_comparison(gender_stats, question_cols)
display_age_comparison(age_stats, question_cols)

# Display full comprehensive table (first few rows)
print("COMPREHENSIVE STATISTICAL TABLE (First 20 rows)")
print("=" * 100)
print(comprehensive_table.head(20).to_string(index=False))
print()

# Export to Excel
print("Exporting comprehensive table to Excel...")
with pd.ExcelWriter('Comprehensive_Statistics_Table_hey.xlsx', engine='openpyxl') as writer:
    comprehensive_table.to_excel(writer, sheet_name='All_Statistics', index=False)
    
    # Create separate sheets for different views
    population_only = comprehensive_table[comprehensive_table['Group'] == 'Population']
    population_only.to_excel(writer, sheet_name='Population_Only', index=False)
    
    gender_only = comprehensive_table[comprehensive_table['Group'].str.contains('Gender')]
    gender_only.to_excel(writer, sheet_name='Gender_Statistics', index=False)
    
    age_only = comprehensive_table[comprehensive_table['Group'].str.contains('Age')]
    age_only.to_excel(writer, sheet_name='Age_Statistics', index=False)

print("Export completed! File saved as 'Comprehensive_Statistics_Table_hey.xlsx'")

POPULATION-LEVEL STATISTICS SUMMARY
Determinant  Count  Mean Median Mode  Q1  Q3 Variance Std_Dev Skewness Kurtosis Entropy Inter_Rater_Agreement RWG_Agreement
         D1     89 5.506    6.0    7 4.0 7.0    2.162   1.470   -0.744   -0.172   1.603                 -0.0%         0.460
         D2     89 5.247    5.0    6 4.0 6.0    2.256   1.502   -0.710    0.041   1.676                 -0.0%         0.436
         D3     90 4.989    5.0    7 4.0 6.0    2.753   1.659   -0.487   -0.595   1.744                 -0.0%         0.312
         D4     88 5.057    5.0    6 4.0 6.0    2.261   1.504   -0.525   -0.325   1.726                 -0.0%         0.435
         D5     90 5.789    6.0    7 5.0 7.0    2.079   1.442   -1.007    0.214   1.452                 -0.0%         0.480
         D6     86 4.279    4.0    4 3.0 6.0    3.239   1.800   -0.192   -0.951   1.883                 -0.0%         0.190
         D7     86 5.372    6.0    7 4.0 7.0    3.483   1.866   -0.945   -0.206   1.604         

## Statistical Difference between groups

In [9]:
def perform_mann_whitney_tests(df, question_cols):
    """Perform Mann-Whitney U tests for gender and age group differences"""
    
    # Create age groups: 12-14, 15-17
    df['Age_Group_2'] = pd.cut(df['Age'], bins=[11, 14, 17], labels=['12-14', '15-17'])
    
    # Initialize results storage
    gender_results = []
    age_results = []
    
    print("Performing Mann-Whitney U tests...")
    print("=" * 60)
    
    # Test gender differences for each determinant
    for col in question_cols:
        # Gender test
        male_data = df[df['Gender'] == 'Male'][col].dropna()
        female_data = df[df['Gender'] == 'Female'][col].dropna()
        
        if len(male_data) > 0 and len(female_data) > 0:
            u_stat, p_value = stats.mannwhitneyu(male_data, female_data, alternative='two-sided')
            cliffs_delta_val, delta_interpretation = cliffs_delta(male_data, female_data)
            gender_results.append({
                'Determinant': col,
                'Male_Count': len(male_data),
                'Female_Count': len(female_data),
                'Male_Mean': male_data.mean(),
                'Female_Mean': female_data.mean(),
                'U_Statistic': u_stat,
                'P_Value': p_value,
                'Cliffs_Delta': cliffs_delta_val,
                'Significant': p_value < 0.05,
                'Effect_Size': 'Large' if p_value < 0.001 else 'Medium' if p_value < 0.01 else 'Small' if p_value < 0.05 else 'None'
            })
        else:
            gender_results.append({
                'Determinant': col,
                'Male_Count': len(male_data),
                'Female_Count': len(female_data),
                'Male_Mean': male_data.mean() if len(male_data) > 0 else np.nan,
                'Female_Mean': female_data.mean() if len(female_data) > 0 else np.nan,
                'U_Statistic': np.nan,
                'P_Value': np.nan,
                'Significant': False,
                'Effect_Size': 'Insufficient Data'
            })
    
    # Test age group differences for each determinant
    for col in question_cols:
        # Age group test
        young_data = df[df['Age_Group_2'] == '12-14'][col].dropna()
        old_data = df[df['Age_Group_2'] == '15-17'][col].dropna()
        
        if len(young_data) > 0 and len(old_data) > 0:
            u_stat, p_value = stats.mannwhitneyu(young_data, old_data, alternative='two-sided')
            cliffs_delta_val, delta_interpretation = cliffs_delta(young_data, old_data)
            age_results.append({
                'Determinant': col,
                'Young_Count': len(young_data),
                'Old_Count': len(old_data),
                'Young_Mean': young_data.mean(),
                'Old_Mean': old_data.mean(),
                'U_Statistic': u_stat,
                'P_Value': p_value,
                'Significant': p_value < 0.05,
                'Cliffs_Delta': cliffs_delta_val,
                'Effect_Size': 'Large' if p_value < 0.001 else 'Medium' if p_value < 0.01 else 'Small' if p_value < 0.05 else 'None'
            })
        else:
            age_results.append({
                'Determinant': col,
                'Young_Count': len(young_data),
                'Old_Count': len(old_data),
                'Young_Mean': young_data.mean() if len(young_data) > 0 else np.nan,
                'Old_Mean': old_data.mean() if len(old_data) > 0 else np.nan,
                'U_Statistic': np.nan,
                'P_Value': np.nan,
                'Significant': False,
                'Effect_Size': 'Insufficient Data'
            })
    
    return pd.DataFrame(gender_results), pd.DataFrame(age_results)

def display_significant_differences(gender_df, age_df):
    """Display determinants with statistically significant differences"""
    
    print("STATISTICALLY SIGNIFICANT DIFFERENCES (p < 0.05)")
    print("=" * 80)
    
    # Gender differences
    significant_gender = gender_df[gender_df['Significant'] == True]
    if len(significant_gender) > 0:
        print(f"\nGENDER DIFFERENCES - {len(significant_gender)} significant determinants:")
        print("-" * 60)
        for _, row in significant_gender.iterrows():
            print(f"{row['Determinant']}: U={row['U_Statistic']:.1f}, p={row['P_Value']:.4f}")
            print(f"  Male: {row['Male_Mean']:.3f} (n={row['Male_Count']})")
            print(f"  Female: {row['Female_Mean']:.3f} (n={row['Female_Count']})")
            print(f"  Effect Size: {row['Effect_Size']}")
            print()
    else:
        print("\nGENDER DIFFERENCES: No statistically significant differences found.")
    
    # Age group differences
    significant_age = age_df[age_df['Significant'] == True]
    if len(significant_age) > 0:
        print(f"\nAGE GROUP DIFFERENCES - {len(significant_age)} significant determinants:")
        print("-" * 60)
        for _, row in significant_age.iterrows():
            print(f"{row['Determinant']}: U={row['U_Statistic']:.1f}, p={row['P_Value']:.4f}")
            print(f"  12-14: {row['Young_Mean']:.3f} (n={row['Young_Count']})")
            print(f"  15-17: {row['Old_Mean']:.3f} (n={row['Old_Count']})")
            print(f"  Effect Size: {row['Effect_Size']}")
            print()
    else:
        print("\nAGE GROUP DIFFERENCES: No statistically significant differences found.")

def create_summary_statistics(gender_df, age_df):
    """Create summary of statistical test results"""
    
    print("SUMMARY OF STATISTICAL TESTS")
    print("=" * 60)
    
    # Gender summary
    total_gender_tests = len(gender_df)
    significant_gender = len(gender_df[gender_df['Significant'] == True])
    print(f"Gender Tests: {significant_gender}/{total_gender_tests} significant ({(significant_gender/total_gender_tests)*100:.1f}%)")
    
    # Age summary
    total_age_tests = len(age_df)
    significant_age = len(age_df[age_df['Significant'] == True])
    print(f"Age Group Tests: {significant_age}/{total_age_tests} significant ({(significant_age/total_age_tests)*100:.1f}%)")
    
    # Effect size distribution
    print(f"\nEffect Size Distribution (Gender):")
    effect_sizes_gender = gender_df['Effect_Size'].value_counts()
    for effect, count in effect_sizes_gender.items():
        print(f"  {effect}: {count}")
    
    print(f"\nEffect Size Distribution (Age):")
    effect_sizes_age = age_df['Effect_Size'].value_counts()
    for effect, count in effect_sizes_age.items():
        print(f"  {effect}: {count}")

In [10]:
# Perform Mann-Whitney U tests
print("Running Mann-Whitney U tests for gender and age group differences...")
gender_results_df, age_results_df = perform_mann_whitney_tests(df, question_cols)

# Display results
print("\nGENDER DIFFERENCES - Mann-Whitney U Test Results")
print("=" * 80)
print(gender_results_df.to_string(index=False))

print("\n\nAGE GROUP DIFFERENCES - Mann-Whitney U Test Results")
print("=" * 80)
print(age_results_df.to_string(index=False))

# Display significant differences
display_significant_differences(gender_results_df, age_results_df)

# Create summary
create_summary_statistics(gender_results_df, age_results_df)

Running Mann-Whitney U tests for gender and age group differences...
Performing Mann-Whitney U tests...

GENDER DIFFERENCES - Mann-Whitney U Test Results
Determinant  Male_Count  Female_Count  Male_Mean  Female_Mean  U_Statistic  P_Value  Cliffs_Delta  Significant Effect_Size
         D1          45            44   5.577778     5.431818       1092.0 0.389620      0.103030        False        None
         D2          45            44   5.622222     4.863636       1320.0 0.005611      0.333333         True      Medium
         D3          45            45   5.222222     4.755556       1212.5 0.100300      0.197531        False        None
         D4          45            43   5.355556     4.744186       1160.0 0.101895      0.198966        False        None
         D5          45            45   5.911111     5.666667       1110.5 0.403128      0.096790        False        None
         D6          43            43   4.674419     3.883721       1158.5 0.040642      0.253110         Tr